In [1]:
function log1p_series_shifted(x)

    left = sqrt(2.)/2 - 1
    right = sqrt(2.) - 1
    sum = 0

    if x < left
        sum -= log(2.) / 2
        x = 1 + 2 * x;
    elseif right < x
        sum += log(2.) / 2
        x = (x - 1) / 2
    end
  
    y = x / (2. + x)
    y2 = y*y
    sum += y
    y *= y2
    sum += y / 3
    y *= y2
    sum += y / 5
    y *= y2
    sum += y / 7
    
    2 * sum
end


log1p_series_shifted (generic function with 1 method)

In [2]:
# E2work is a voigt (size 6)
function computeDetCM1(E2work) 

    E2work[1]*(E2work[2]*E2work[3]-E2work[4]*E2work[4]) +
    E2work[6]*(E2work[5]*E2work[4]-E2work[6]*E2work[3]) +
    E2work[5]*(E2work[6]*E2work[4]-E2work[5]*E2work[2]) +
    E2work[1] + E2work[2] + E2work[3] +
    E2work[1]*E2work[2] + E2work[1]*E2work[3] +
    E2work[2]*E2work[3] - E2work[6]*E2work[6] -
    E2work[5]*E2work[5] - E2work[4]*E2work[4]
end

computeDetCM1 (generic function with 1 method)

In [3]:
function computeMatinvSym(A, detA, Ainv)
  
  # Compute A^(-1) : A-Inverse
  B = [ A[2,2]*A[3,3] - A[2,3]*A[3,2],
        A[1,1]*A[3,3] - A[1,3]*A[3,1],
        A[1,1]*A[2,2] - A[1,2]*A[2,1],
        A[1,3]*A[2,1] - A[1,1]*A[2,3],
        A[1,2]*A[2,3] - A[1,3]*A[2,2], 
        A[1,3]*A[3,2] - A[1,2]*A[3,3]
      ]

  for m in 1:6
    Ainv[m] = B[m] / (detA)
  end

end

computeMatinvSym (generic function with 1 method)

In [4]:
function computeS!(E2work, Swork, lambda, mu)

      E2 = [ 
              E2work[1] E2work[6] E2work[5];
              E2work[6] E2work[2] E2work[4];
              E2work[5] E2work[4] E2work[3]
           ]
   
      C = [ 
            1+E2[1,1] E2[1,2]   E2[1,3];
            E2[1,2]   1+E2[2,2] E2[2,3];
            E2[1,3]   E2[2,3]   1+E2[3,3]
         ]

      Cinvwork = zeros(6)
      detCm1 = computeDetCM1(E2work)
      computeMatinvSym(C, detCm1+1, Cinvwork)
      
      C_inv = [ 
                Cinvwork[1] Cinvwork[6] Cinvwork[5];
                Cinvwork[6] Cinvwork[2] Cinvwork[4];
                Cinvwork[5] Cinvwork[4] Cinvwork[3]
              ]

      logJ = log1p_series_shifted(detCm1) / 2.

      indj = [1 2 3 2 1 1]
      indk = [1 2 3 3 3 2]
      Swork = zeros(6)
      for m in 1:6
            Swork[m] = lambda*logJ*Cinvwork[m]
            for n in 1:3
                  Swork[m] += mu*C_inv[indj[m], n]*E2[n, indk[m]]
            end
      end
      nothing
end


computeS! (generic function with 1 method)

In [5]:
# Run an example
E = .3
nu = .3
TwoMu = E / (1 + nu)
mu = TwoMu / 2
Kbulk = E / (3*(1 - 2*nu)) # Bulk Modulus
lambda = (3*Kbulk - TwoMu) / 3
E2work = [
    0.5895232828911128,
    0.2362491738162759,
    0.9793730522395296,
    0.2190993957421843,
    0.0126503210747925,
    0.6570956167695403
]
detCm1 = computeDetCM1(E2work)

1.9620580003448598

In [6]:
Swork = zeros(6)
computeS!(E2work, Swork, lambda, mu)

In [7]:
∂z_∂E2work = zeros(6)
∂z_∂Swork = fill!((similar(Swork)), 1)

6-element Vector{Float64}:
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0

In [8]:
using Zygote
Zygote.pullback(*, E2work, lambda, mu)[2](∂z_∂Swork )[1]

6-element Vector{Float64}:
 0.019970414201183423
 0.019970414201183423
 0.019970414201183423
 0.019970414201183423
 0.019970414201183423
 0.019970414201183423